In [7]:
#@title 🚀 Yannova 3D Generator - Run this cell!
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate trimesh flask flask-cors pyngrok
!pip install -q git+https://github.com/openai/shap-e.git

import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import trimesh, base64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"GPU: {torch.cuda.is_available()}")

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

app = Flask(__name__)
CORS(app)

@app.route('/generate', methods=['POST'])
def generate():
    prompt = request.json.get('prompt', 'a door')
    latents = sample_latents(batch_size=1, model=model, diffusion=diffusion, guidance_scale=15.0, model_kwargs=dict(texts=[prompt]), progress=True, clip_denoised=True, use_fp16=True, use_karras=True, karras_steps=64)
    mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()
    tri = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)
    return jsonify({"success": True, "model": base64.b64encode(tri.export(file_type='glb')).decode()})

@app.route('/health')
def health():
    return jsonify({"status": "ok"})

# Vervang dit met je eigen authtoken van https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("JOUW_NGROK_TOKEN")
url = ngrok.connect(5000)
print(f"\n🌐 URL: {url}\n")
app.run(port=5000)

^C
Traceback (most recent call last):
  File "/Users/innovarslabo/Desktop/yannova/.venv/bin/pip", line 3, in <module>
    from pip._internal.cli.main import main
  File "/Users/innovarslabo/Desktop/yannova/.venv/lib/python3.14/site-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/Users/innovarslabo/Desktop/yannova/.venv/lib/python3.14/site-packages/pip/_internal/cli/autocompletion.py", line 12, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/Users/innovarslabo/Desktop/yannova/.venv/lib/python3.14/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/Users/innovarslabo/Desktop/yannova/.venv/lib/python3.14/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/Users/innovarslabo/Desktop/yannova/.venv/lib/python3.14/site

ModuleNotFoundError: No module named 'ipywidgets'